Ej. 1. Indexado de una colección estandarizada

Primero instalamos todo el paquete BM25s ejecutando la siguiente orden:

In [ ]:
!pip install bm25s[full]

Ahora importamos las librerías Python necesarias para trabajar

In [ ]:
import bm25s # To create indexes and search documents using BM25
import Stemmer  # For stemming terms
import json  # To load the JSON-formatted corpus

Descargamos la colección TREC-COVID

In [ ]:
!gdown 1Q1ZnHkstIIQQF8NTPyKQH7sEcpVfThFO
!unzip -o trec-covid-RI.zip

Y comenzamos el ejercicio con la preparación del corpus.

Inicialmente partí del código del primer Notebook de las prácticas pero adapté el código para que pudiera trabajar con el material del entregable.
El fichero a parsear, y del que partimos para la recopilación de los documentos con los que crear el índice, es un fichero JSONL en lugar de un esperado JSON.

Esto significa que el corpus.jsonl es un fichero de texto en el que cada línea es un objeto JSON, luego mi idea fue leer y recorrer el fichero línea a línea y por cada línea de dicho fichero generar el JSON que representa y guardarlo en una lista de JSONs con los que trabajar a posteriori.

Luego adapté el código del primer Notebook para que se recogieran los campos que bm25 espera (id, title y text).

In [ ]:
# Parseamos el corpus
with open("corpus.jsonl", "r", encoding="utf-8") as f:
    corpus_lines = f.readlines() # Leemos todas las lineas del JSONL
    corpus_content = [] # Preparamos una lista de objetos JSON
    for linea in corpus_lines: # Cada línea de un JSONL es un objeto JSON...
        corpus_content.append(json.loads(linea)) # Obtenemos el objeto JSON de la línea i-ésima del JSONL y lo almacenamos en la lista de JSONs

# Preparamos el corpus en los dos formatos requeridos
corpus_verbatim = list() # Una versión literal que mantenga la estructura original de los "documentos".
corpus_plaintext = list() # Una versión de texto plano para tokenización e indexación.
for entry in corpus_content:
    # BM25S expects 'id', 'title', and 'text'
    document = {"id": entry["_id"], "title": entry["title"].lower(), "text": entry["text"].lower()}
    corpus_verbatim.append(document)
    corpus_plaintext.append(entry["text"].lower())

Implementé la funcionalidad para la tokenización mediante un stemmer y basándome en el código del primer Notebook.

In [ ]:
# Tokenizamos mediante un stemmer para el idioma inglés (PREPROCESADO Y NORMALIZACIÓN)
stemmer = Stemmer.Stemmer("english")
corpus_tokenized = bm25s.tokenize(corpus_plaintext, stopwords="en", stemmer=stemmer, show_progress=True)

El siguiente paso a seguir en el ejercicio es crear el retriever BM25S e indexar la colección.


Implementamos la creación del índice a partir del código del primer Notebook adaptándolo a mi caso de uso.

In [ ]:
bm25_flavor = "lucene" # Usamos la implementacion lucene de la funcion del IDF
idf_flavor = "lucene" # Usamos la implementacion lucene de la funcion del IDF

retriever = bm25s.BM25(corpus=corpus_verbatim, method=bm25_flavor, idf_method=idf_flavor)
retriever.index(corpus_tokenized, show_progress=True) # Indexamos el corpus tokenizado

retriever.save("TRECS", corpus=corpus_verbatim) # Guardamos el index generado para poder reutilizarlo llamándolo "trecs"

Una vez creado el índice y guardado en nuestro espacio de trabajo procedemos a realizar la búsqueda con BM25s.

Las funciones que cargan los juicios de relevancia y las queries son adaptadas de los notebooks para que puedan trabajar con el formato de los ficheros a procesar.

Procedemos con la implementación del envío y carga de las queries:

In [ ]:
# Cargamos el índice (no cargamos el corpus para evitar la excepción del encoding [hablado por correo con Dani])
retriever = bm25s.BM25.load("TRECS")

# Enviamos consultas
queries = []
with open("queries.jsonl", "r", encoding="utf-8") as f:
    queries_lines = f.readlines() # Leemos todas las lineas del JSONL
    for linea in queries_lines: # Cada línea de un JSONL es un objeto JSON...
        queries.append(json.loads(linea)) # Obtenemos el objeto JSON de la línea i-ésima del JSONL y lo almacenamos en la lista de JSONs

# Función para cargar consultas desde un fichero json para enviar a una
# colección dada y obtener listas de resultados.
def submit_queries_and_get_run(queries, stemmer, retriever, max_results=10):
    run = {}

    # Iteramos para enviar cada consulta al retriever para obtener
    # resultados
    for query in queries:
        # Para evaluar después el rendimiento, es útil conservar el id de
        # la consulta
        query_id = query["_id"]

        query_string = query["metadata"]["narrative"].lower()

        # Tokenizamos la consulta. ¡Atención! Debemos tokenizar la consulta
        # con la misma configuración que usamos al indexar el corpus. En
        # este caso, eliminando las palabras vacías del inglés y aplicando
        # el stemmer de inglés
        #
        query_tokenized = bm25s.tokenize(query_string, stopwords="en", stemmer=stemmer, show_progress=False)

        # Retornamos los k resultados más “top” como una tupla con nombre.
        # Por favor, lee la documentación de BM25S para otras alternativas.
        #
        results = retriever.retrieve(query_tokenized, corpus=retriever.corpus, k=max_results, return_as="tuple", show_progress=False)

        # Los documentos y sus puntuaciones se guardan en dos campos
        # diferentes de la tupla.
        #
        returned_documents = results.documents[0]
        relevance_scores = results.scores[0]

        returned_ids = []
        for i in range(len(returned_documents)):
##            print(f"\t{i}\t{relevance_scores[i]}\t{returned_documents[i]['id']}\t{returned_documents[i]['title'][0:80]}...")
            returned_ids.append(str(returned_documents[i]))
        run[query_id] = returned_ids

    return run

original_run = submit_queries_and_get_run(queries, stemmer, retriever)
print(original_run['1'])

Y ahora con la carga de los juicios de relevancia:

In [ ]:
def cargarJuiciosDeRelevancia(recursoJuiciosRelevancia):
    # Leer los datos del archivo de texto
    juicios = {}
    saltarLinea = True
    with open(recursoJuiciosRelevancia, 'r') as f:
        for linea in f:
            if saltarLinea:
                saltarLinea = False
                continue
            if linea.strip():  # Saltar líneas vacías
                query_id, corpus_id, score = linea.split()
                if query_id not in juicios:
                    juicios[query_id] = []
                juicios[query_id].append({"corpus_id": corpus_id, "score": int(score)})

    # El formato de los juicios de relevancia necesita ser ligeramente
    # transformado
    relevance_judgements_reformat = dict()
    for query_id in juicios:
        rel_docs = []
        for j in juicios[query_id]:
            rel_docs.append(j["corpus_id"])
        relevance_judgements_reformat[query_id] = rel_docs

    return relevance_judgements_reformat

juiciosRelevancia = cargarJuiciosDeRelevancia("qrels.tsv")
for i in juiciosRelevancia:
    print(i, " ", juiciosRelevancia[i])